In [10]:
import pandas as pd

df = pd.read_csv("../data/tmdb_crawled_fast.csv")

print(df.shape)
df.head()


(2200, 13)


,tmdb_id,title,overview,release_date,genres,runtime,vote_average,vote_count,popularity,cast_top5,director,keywords,poster_path
0,83533,Avatar: Fire and Ash,In the wake of the devastating war against the...,2025-12-17,"['Science Fiction', 'Adventure', 'Fantasy']",198,7.356,663,566.0459,"['Sam Worthington', 'Zoe Saldaña', 'Sigourney ...",James Cameron,"['witch', 'clone', 'space war', 'tribe', 'sequ...",/g96wHxU7EnoIFwemb2RgohIXrgW.jpg
1,1247002,The Old Woman with the Knife,"Aging assassin Hornclaw has seen it all, but s...",2025-04-30,"['Action', 'Mystery', 'Drama']",123,6.588,41,254.9502,"['Lee Hye-young', 'Kim Sung-cheol', 'Yeon Woo-...",Min Kyu-dong,['based on novel or book'],/wbmxnsv41vsg5UEaNDlf203dOWw.jpg
2,1084242,Zootopia 2,After cracking the biggest case in Zootopia's ...,2025-11-26,"['Animation', 'Comedy', 'Adventure', 'Family',...",107,7.633,742,340.9615,"['Ginnifer Goodwin', 'Jason Bateman', 'Ke Huy ...",Jared Bush,"['snake', 'bunny', 'fox', 'cop', 'sequel', 'an...",/bjUWGw0Ao0qVWxagN3VCwBJHVo6.jpg
3,425274,Now You See Me: Now You Don't,The original Four Horsemen reunite with a new ...,2025-11-12,"['Thriller', 'Crime', 'Mystery']",112,6.500,640,298.8916,"['Jesse Eisenberg', 'Dominic Sessa', 'Ariana G...",Ruben Fleischer,"['diamond', 'sequel', 'illusionist', 'team up'...",/oD3Eey4e4Z259XLm3eD3WGcoJAh.jpg
4,982843,The Great Flood,When a raging flood traps a researcher and her...,2025-09-18,"['Science Fiction', 'Adventure', 'Drama']",109,6.145,259,493.8003,"['Kim Da-mi', 'Park Hae-soo', 'Kwon Eun-seong'...",Kim Byung-woo,"['climate change', 'natural disaster', 'global...",/1tUOZQDgZaGqZtrB21MieiXARL2.jpg


In [11]:
# trùng theo tmdb_id
df = df.drop_duplicates(subset=["tmdb_id"])

# trùng theo title (phòng khi ID lỗi)
df = df.drop_duplicates(subset=["title"])

print("After deduplicate:", df.shape)


After deduplicate: (2173, 13)


In [12]:
# Overview rỗng → chuỗi rỗng
df["overview"] = df["overview"].fillna("")

# runtime → median
df["runtime"] = pd.to_numeric(df["runtime"], errors="coerce")
df["runtime"] = df["runtime"].fillna(df["runtime"].median())

# director → Unknown
df["director"] = df["director"].fillna("Unknown")

# keywords / cast nếu rỗng → []
df["keywords"] = df["keywords"].fillna("[]")
df["cast_top5"] = df["cast_top5"].fillna("[]")


In [13]:
from sklearn.preprocessing import MinMaxScaler

num_cols = ["vote_average", "popularity", "runtime"]

df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


In [14]:
df.shape

(2173, 13)

In [15]:
import ast

def clean_list(x):
    try:
        lst = ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return []
    return [i.strip().lower().replace(" ", "_") for i in lst if isinstance(i, str)]

df["genres"] = df["genres"].apply(clean_list)
df["keywords"] = df["keywords"].apply(clean_list)
df["cast_top5"] = df["cast_top5"].apply(clean_list)

df["director"] = df["director"].apply(lambda x: x.lower().replace(" ", "_") if isinstance(x, str) else "")


In [16]:
def make_soup(row):
    return " ".join(
        row["genres"] +
        row["keywords"] +
        row["cast_top5"] +
        [row["director"]] +
        [row["overview"].lower()]
    )

df["soup"] = df.apply(make_soup, axis=1)


In [17]:
df.to_csv("/home/anonymous/code/KHDL/btl/data/tmdb_cleaned.csv", index=False, encoding="utf-8-sig")
print("Saved tmdb_cleaned.csv")


Saved tmdb_cleaned.csv
